### 05 - PyTorch going modular

Going from notebooks to scripts

#### Get data

In [2]:
import os
import requests
import zipfile
from pathlib import Path

# setup path for data folder
data_path = Path('data')
image_path = data_path / "pizza_steak_sushi"

# If the image folder doesn't exist, download and extract the data
if image_path.is_dir():
    print(f"{image_path} directory already exists, skipping download...")
else:
    print(f"Did not find {image_path} directory, creating one...")
    image_path.mkdir(parents=True, exist_ok=True)
    # Download pizza, steak and sushi data
    with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
        request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
        print(f"Downloading {len(request.content)} MB of data for pizza, steak and sushi images...")
        f.write(request.content)

    # Unzip the downloaded file
    with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", 'r') as zip_ref:
        zip_ref.extractall(image_path)
        print(f"Unzipped pizza, steak and sushi data to {image_path}")

    # Remove the zip file
    os.remove(data_path / "pizza_steak_sushi.zip")

data\pizza_steak_sushi directory already exists, skipping download...


In [4]:
%%writefile going_modular/data_setup.py
"""
Contains functionality for creating PyTorch DataLoaders for
image classification data
"""
import os

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

NUM_WORKERS = os.cpu_count()

def create_dataloaders(train_dir: str, 
                       test_dir: str, 
                       transform: transforms.Compose, 
                       batch_size: int, 
                       num_workers: int = NUM_WORKERS):
    """
    Creates training and testing DataLoaders.

    Takes in training and testing directory paths and turns them into PyTorch
    Datasets and then into DataLoaders.

    Args:
        train_dir: Path to training directory.
        test_dir: Path to testing directory.
        transform: torchvision.transforms.Compose containing transformations to perform on data.
        batch_size: Number of samples per batch of data.
        num_workers: Number of workers for DataLoader to use.

    Returns:
        A tuple of (train_dataloader, test_dataloader, class_names).
    """
    train_data = datasets.ImageFolder(train_dir, transform=transform)
    test_data = datasets.ImageFolder(test_dir, transform=transform)
    class_names = train_data.classes
    train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=num_workers)
    return train_dataloader, test_dataloader, class_names

Writing going_modular/data_setup.py


In [6]:
%%writefile going_modular/model_builder.py
"""
Contains PyTorch model code to instantiate a TinyVGG model
"""
import torch
from torch import nn

class TinyVGG(nn.Module):
    """
    Creates a TinyVGG model.
    """
    def __init__(self, input_shape: int, hidden_shape: int, output_shape: int):
        super().__init__()
        self.convblock1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape, out_channels=hidden_shape, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_shape, out_channels=hidden_shape, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        )
        self.convblock2 = nn.Sequential(
            nn.Conv2d(in_channels=hidden_shape, out_channels=hidden_shape, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_shape, out_channels=hidden_shape, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden_shape * 16 * 16, out_features=output_shape)
        )

    def forward(self, x):
        x = self.convblock1(x)
        x = self.convblock2(x)
        x = self.classifier(x)
        return x

Writing going_modular/model_builder.py


In [7]:
%%writefile going_modular/engine.py
"""
Contains functions for training and testing a PyTorch model
"""
import torch
from tqdm import tqdm
from typing import Dict, Tuple, List

def train_step(model: torch.nn.Module, 
                dataloader: torch.utils.data.DataLoader, 
                loss_fn: torch.nn.Module, 
                optimizer: torch.optim.Optimizer, 
                device: str) -> Tuple[float, float]:
    """
    Trains a PyTorch model for a single epoch.

    Args:
        model: A PyTorch model to be trained.
        dataloader: A DataLoader containing the training data.
        loss_fn: A loss function to compute the loss.
        optimizer: An optimizer to update the model's parameters.
        device: The device to use for training (e.g., "cuda" or "cpu").

    Returns:
        A tuple of (train_loss, train_accuracy).
    """
    model.train()
    train_loss, train_acc = 0, 0
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        y_pred = model(X)
        loss = loss_fn(y_pred, y)
        train_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        y_pred_class = torch.argmax(y_pred, dim=1)
        train_acc += (y_pred_class == y).sum().item()
    train_loss /= len(dataloader)
    train_acc /= len(dataloader.dataset)
    return train_loss, train_acc

def test_step(model: torch.nn.Module, 
              dataloader: torch.utils.data.DataLoader, 
              loss_fn: torch.nn.Module, 
              device: str) -> Tuple[float, float]:
    """ Tests a PyTorch model for a single epoch.
    Args:
        model: A PyTorch model to be tested.
        dataloader: A DataLoader containing the testing data.
        loss_fn: A loss function to compute the loss.
        device: The device to use for testing (e.g., "cuda" or "cpu").
    Returns:
        A tuple of (test_loss, test_accuracy).
    """
    model.eval()
    test_loss, test_acc = 0, 0
    with torch.no_grad():
        for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)
            y_pred = model(X)
            loss = loss_fn(y_pred, y)
            test_loss += loss.item()
            y_pred_class = torch.argmax(y_pred, dim=1)
            test_acc += (y_pred_class == y).sum().item()
    test_loss /= len(dataloader)
    test_acc /= len(dataloader.dataset)
    return test_loss, test_acc

def train(model: torch.nn.Module, 
        train_dataloader: torch.utils.data.DataLoader, 
        test_dataloader: torch.utils.data.DataLoader, 
        optimizer_fn: torch.nn.Module, 
        loss_fn: torch.optim.Optimizer, 
        epochs: int, 
        device: str) -> Dict[str, List[float]]:
    """ Trains and tests a PyTorch model for a specified number of epochs.
    Args:
        model: A PyTorch model to be trained and tested.
        train_dataloader: A DataLoader containing the training data.
        test_dataloader: A DataLoader containing the testing data.
        loss_fn: A loss function to compute the loss.
        optimizer: An optimizer to update the model's parameters.
        epochs: The number of epochs to train the model for.
        device: The device to use for training and testing (e.g., "cuda" or "cpu").
    Returns:
        A dictionary of training and testing losses and accuracies.
    """
    results = {"train_loss": [], "train_acc": [], "test_loss": [], "test_acc": []}
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model, train_dataloader, loss_fn, optimizer, device)
        test_loss, test_acc = test_step(model, test_dataloader, loss_fn, device)
        print(f"Epoch: {epoch+1}/{epochs} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Test Loss: {test_loss:.4f} | Test Acc: {test_acc:.4f}")
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)
    
        print(
            f"Epoch: {epoch+1}/{epochs} | "
            f"Train Loss: {train_loss:.4f} | "
            f"Train Acc: {train_acc:.4f} | "
            f"Test Loss: {test_loss:.4f} | "
            f"Test Acc: {test_acc:.4f}"
            
        )
    return results

Writing going_modular/engine.py


In [8]:
%%writefile going_modular/utils.py
"""
Contains utility functions for saving and loading models
"""
import torch
from pathlib import Path

def save_model(model: torch.nn.Module, 
              target_dir: str, 
              model_name: str):
    """
    Saves a PyTorch model to a target directory.

    Args:
        model: A PyTorch model to be saved.
        target_dir: The directory to save the model to.
        model_name: The name of the model file.
    """
    target_dir_path = Path(target_dir)
    target_dir_path.mkdir(parents=True, exist_ok=True)
    model_path = target_dir_path / model_name
    print(f"Saving model to {model_path}")
    torch.save(obj=model.state_dict(), f=model_path)

Writing going_modular/utils.py


In [10]:
%%writefile going_modular/train.py
"""
Trains a PyTorch model using device agnostic code
"""
import os
import torch
import argparse
from torchvision import transforms
import datasetup, engine, model_builder, utils

if __name__ == "__main__":
    parser.add_argument("--num_epochs", type=int, default=NUM_EPOCHS, help="Number of epochs to train the model for")
    parser.add_argument("--batch_size", type=int, default=BATCH_SIZE, help="Batch size for training and testing")
    parser.add_argument("--hidden_units", type=int, default=HIDDEN_UNITS, help="Number of hidden units in the model")
    parser.add_argument("--learning_rate", type=float, default=LEARNING_RATE, help="Learning rate for the optimizer")
    args = parser.parse_args()

    # Update hyperparameters based on command line arguments
    NUM_EPOCHS = args.num_epochs
    BATCH_SIZE = args.batch_size
    HIDDEN_UNITS = args.hidden_units
    LEARNING_RATE = args.learning_rate
    # Setup directories
    train_dir = "data/pizza_steak_sushi/train"
    test_dir = "data/pizza_steak_sushi/test"
    # Setup device-agnostic code
    device = "cuda" if torch.cuda.is_available() else "cpu"
    # Setup transformations
    transform = transforms.Compose([
        transforms.Resize((64, 64)),
        transforms.ToTensor(),
    ])
    # Create DataLoaders
    train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
        train_dir=train_dir,
        test_dir=test_dir,
        transform=transform,
        batch_size=BATCH_SIZE
    )
    # Setup model
    model = model_builder.TinyVGG(
        input_shape=3,
        hidden_shape=HIDDEN_UNITS,
        output_shape=len(class_names)
    ).to(device)

    # Setup loss, optimizer
    loss_fn = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
    # Train the model
    results = engine.train(
        model=model,
        train_dataloader=train_dataloader,
        test_dataloader=test_dataloader,
        loss_fn=loss_fn,
        optimizer=optimizer,
        epochs=NUM_EPOCHS,
        device=device
    )
    # Save the model
    utils.save_model(
        model=model,
        target_dir="models",
        model_name="tinyvgg_model.pth"
    )

Overwriting going_modular/train.py


In [11]:
%%writefile going_modular/predict.py
"""
Predicts the class of an image using a pre-trained PyTorch model
"""
import torch
from torchvision import transforms
from PIL import Image
from model_builder import TinyVGG
from utils import save_model
from pathlib import Path

def predict(image_path: str, model_path: str, class_names: list):
    """
    Predicts the class of an image using a pre-trained PyTorch model.

    Args:
        image_path: Path to the image to predict.
        model_path: Path to the pre-trained model.
        class_names: List of class names.
    """
    # Load the model
    model = TinyVGG(input_shape=3, hidden_shape=10, output_shape=len(class_names))
    model.load_state_dict(torch.load(model_path))
    model.eval()

    # Setup transformations
    transform = transforms.Compose([
        transforms.Resize((64, 64)),
        transforms.ToTensor(),
    ])

    # Load and transform the image
    image = Image.open(image_path)
    image = transform(image).unsqueeze(0)

    # Make prediction
    with torch.no_grad():
        y_pred = model(image)
        y_pred_class = torch.argmax(y_pred, dim=1)
        predicted_class = class_names[y_pred_class.item()]
        print(f"Predicted class: {predicted_class}")

Writing going_modular/predict.py
